# **Part 1: Load Documents & Execute Reranking Model**

## **1. Install Pinecone libraries**

In [1]:
!pip install pinecone==6.0.1 pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 8.2 MB/s eta 0:00:00


In [2]:
import os
if not os.environ.get("PINECONE_API_KEY"):
   from pinecone_notebooks.colab import Authenticate
   Authenticate()

## **3. Instantiate the Pinecone client**

In [4]:
from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "us-east-1"  # e.g., "us-west1-gcp"
pc = Pinecone(api_key=api_key, environment=environment)

## **4. Define your query & documents**

In [5]:
query = "Tell me about Apple's products"
documents = [
    "Apple Inc. released the new iPhone with improved camera features.",
    "An apple a day keeps the doctor away, according to the old saying.",
    "The MacBook Air is one of Apple’s most popular laptops for students.",
    "I bought some fresh apples from the market to bake a pie.",
    "Apple’s latest keynote announced updates to iOS and new AirPods."
]

## **5. Call the reranker**

In [6]:
from pinecone import RerankModel
reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=query,
   documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
   top_n=1  # e.g., 3
)

## **6. Inspect reranked results**

In [7]:
print(reranked)

RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=2,
    score=0.1859472,
    document={
        id='2',
        text='The MacBook Air is one of Apple’s most popular laptops for students.'
    }
  }],
  usage={'rerank_units': 1}
)


In [8]:
def show_reranked(query, matches):
   print(f"Query: {query}")
   for i, m in enumerate(matches):
       print(f"{i+1} response {m.score} : {m.document.text}")  # Print the position (i+1), m.score, and m.document.text
show_reranked(query, reranked.data)

Query: Tell me about Apple's products
1 response 0.1859472 : The MacBook Air is one of Apple’s most popular laptops for students.


# **Part 2: Setup a Serverless Index for Medical Notes**

## **1. Install data & model libraries**

In [9]:
!pip install --q pandas torch transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00


## **2. Import modules & define environment settings**

In [10]:
import os, time, pandas as pd, torch
from pinecone import Pinecone, ServerlessSpec

cloud = "aws"        # e.g., "aws"
region = "us-east-1"       # e.g., "us-east-1"
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = "pinecone-reranker"

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment=cloud)

## **3. Create or recreate the index**

In [11]:
if pc.has_index(index_name):
   pc.delete_index(index_name)
pc.create_index(
   name=index_name,
   dimension=384,           # must match embedding vector size
   metric="cosine",
   spec=spec
)

{
    "name": "pinecone-reranker",
    "metric": "cosine",
    "host": "pinecone-reranker-soaeuog.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

# **Part 3: Load the Sample Data**

## **1. Download & read JSONL**

In [31]:
import requests, tempfile

with tempfile.TemporaryDirectory() as tmpdir:
   file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
   url = "https://raw.githubusercontent.com/pinecone-io/examples/refs/heads/master/docs/data/sample_notes_data.jsonl"  # raw GitHub URL to JSONL file
   resp = requests.get(url)
   resp.raise_for_status()
   open(file_path, "wb").write(resp.content)
   df = pd.read_json(file_path, orient='records', lines=True)

In [32]:
df.head()

,id,values,metadata
0,P011,"[-0.2027486265, 0.2769146562, -0.1509393603, 0...","{'advice': 'rest, hydrate', 'symptoms': 'heada..."
1,P001,"[0.1842793673, 0.4459365904, -0.0770567134, 0....","{'tests': 'EKG, stress test', 'symptoms': 'che..."
2,P002,"[-0.2040648609, -0.1739618927, -0.2897160649, ...","{'HbA1c': '7.2', 'condition': 'diabetes', 'med..."
3,P003,"[0.1889383644, 0.2924542725, -0.2335938066, -0...","{'symptoms': 'cough, wheezing', 'diagnosis': '..."
4,P004,"[-0.12171068040000001, 0.1674752235, -0.231888...","{'referral': 'dermatology', 'condition': 'susp..."


# **Part 4: Upsert Data into the Index**

## **1. Instantiate index client & upsert**

In [38]:
index = pc.Index(index_name)
index.upsert_from_dataframe(df)

sending upsert requests:   0%|          | 0/100 [00:00<?, ?it/s]

{'upserted_count': 100}

### **2. Wait for availability**

In [39]:
def is_ready(idx):
   stats = idx.describe_index_stats()
   return stats.total_vector_count > 0

while not is_ready(index):
   time.sleep(5)
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 10099}},
 'total_vector_count': 10099,
 'vector_type': 'dense'}


# **Part 5: Embedding Function**

## **1. Define your embedding function**

In [43]:
from sentence_transformers import SentenceTransformer

def get_embedding(text):
   model = SentenceTransformer("all-MiniLM-L6-v2")  # e.g., "all-MiniLM-L6-v2"
   return model.encode(text).tolist()


## **2. Run a semantic search query**

In [44]:
question = "what if my patient has leg pain"
emb = get_embedding(question)
results = index.query(vector=emb, top_k=3, include_metadata=True)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)

In [45]:
print(matches)

[{'id': 'P0100',
 'metadata': {'advice': 'over-the-counter pain relief, stretching',
              'symptoms': 'muscle pain'},
 'score': 0.518134952,
 'values': []}, {'id': 'P047',
 'metadata': {'symptoms': 'back pain', 'treatment': 'physical therapy'},
 'score': 0.50096035,
 'values': []}, {'id': 'P095',
 'metadata': {'symptoms': 'back pain', 'treatment': 'physical therapy'},
 'score': 0.50096035,
 'values': []}]



# **Part 6: Display & Rerank Clinical Notes**

## **1. Display initial search results**

In [46]:
def show_results(q, matches):
   print(f"Question: {q}")
   for i, m in enumerate(matches):
       print(f"response {i+1} : {m.id}, {m.score}, {m.metadata}")
show_results(question, matches)

Question: what if my patient has leg pain
response 1 : P0100, 0.518134952, {'advice': 'over-the-counter pain relief, stretching', 'symptoms': 'muscle pain'}
response 2 : P047, 0.50096035, {'symptoms': 'back pain', 'treatment': 'physical therapy'}
response 3 : P095, 0.50096035, {'symptoms': 'back pain', 'treatment': 'physical therapy'}


## **2. Prepare documents for reranking**

In [47]:
rerank_docs = [
   {"id": m.id, "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])}
   for m in matches
]
rerank_query = "What is the most likely cause of leg pain?"

In [54]:
print(rerank_docs)

[{'id': 'P0100', 'reranking_field': 'advice: over-the-counter pain relief, stretching; symptoms: muscle pain'}, {'id': 'P047', 'reranking_field': 'symptoms: back pain; treatment: physical therapy'}, {'id': 'P095', 'reranking_field': 'symptoms: back pain; treatment: physical therapy'}]


## **3. Execute serverless reranking**



In [57]:
reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=rerank_query,
   documents=rerank_docs,
   rank_fields=["reranking_field"],
   top_n=1
)

In [60]:
print("Reranked result:", reranked)

Reranked result: RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=0,
    score=0.0008040859,
    document={
        id='P0100',
        reranking_field='advice: over-the-counter pain relief, stretching; symptoms: muscle pain'
    }
  }],
  usage={'rerank_units': 1}
)


## **4. Show reranked results**

In [61]:
def show_reranked(q, matches):
   print(f"Refined Query: {q}")
   for i, m in enumerate(matches):
       print(f"response {i+1} : {m.document.id}, {m.score}, {m.document.reranking_field}")
show_reranked(rerank_query, reranked.data)

Refined Query: What is the most likely cause of leg pain?
response 1 : P0100, 0.0008040859, advice: over-the-counter pain relief, stretching; symptoms: muscle pain
